# Prepration
Loading env variables and vectorDB

In [1]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [2]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

## Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [3]:
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numex

In [4]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func=tools[0].run,
        description="Tool to calculate, nothing else"
    )
]

In [5]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)
agent.run("How are you?")



> Entering new  chain...
 This is not a math question, so I cannot use the Math Tool.
Action: None
Action Input: None
Observation: None is not a valid tool, try another one.
Thought: I should use a different tool to answer this question.
Final Answer: I'm doing well, thank you for asking!

> Finished chain.


"I'm doing well, thank you for asking!"

In [6]:
agent.run("What is 100 devided by 25?")



> Entering new  chain...
 I need to divide two numbers.
Action: Math Tool
Action Input: 100/25
Observation: Answer: 4.0
Thought: I now know the final answer.
Final Answer: 100 divided by 25 is equal to 4.0.

> Finished chain.


'100 divided by 25 is equal to 4.0.'

# Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [7]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [8]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [9]:
agent.run("When does the restaurant open?") 



> Entering new  chain...
 I need to find out the opening hours.
Action: restaurant search
Action Input: opening hours
Observation: Q: What are the hours of operation for your restaurant?
A: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.

Q: What type of cuisine does your restaurant serve?
A: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.
Q: Do you have any specials or promotions running currently?
A: Yes, we have a Happy Hour promotion from 3 to 5 p.m. on weekdays, with special prices on selected appetizers and drinks.
Q: Do you offer vegetarian or vegan options?
A: Yes, we have a range of dishes to cater to vegetarians and vegans. Please let our staff know about any dietary restrictions you have when you order.
Q: What steps is your restaurant taking to ensure safety amid the ongoing pandemic?
A: We adhere to strict health and safety protoco

'Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.'